## Exploring Large Language Models: LLaMA and Mistral

In this notebook, we will dive into two famous large language models, LLaMA and Mistral, along with their instruction-tuned versions. We'll explore how each model performs on various tasks, with a particular focus on generating structured responses in JSON format.

These models have been fine-tuned to follow instructions, making them suitable for a range of NLP applications. Through this lab, you will:

- Learn how to load and interact with LLaMA and Mistral models using the `pipeline` and `chat_template` functions.
- Examine the performance of their instruction-based variants.
- Generate structured outputs, specifically in JSON, for practical applications.

> **Disclaimer**: Before starting this lab, ensure you have requested access to the required models on Hugging Face and have logged in to your Hugging Face account. Access is necessary for the following models:
>
> - [Llama-3.2-1B](https://huggingface.co/meta-llama/Llama-3.2-1B)
> - [Llama-3.2-1B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct)
> - [Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2)
>
> You can log in to Hugging Face directly from this notebook using the provided code snippet.


In [1]:
from huggingface_hub import login

import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [2]:
# Login to the Hugging Face model hub to be able to upload models
with open("../hf_token.txt", "r") as f:
    token = f.read()
    f.close()

login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/fgiobergia/.cache/huggingface/token
Login successful


# 1. LLaMA

In this part of the lab, we will explore **LLaMA (Large Language Model Meta AI)**, which is one of the most known large language models developed by Meta (Facebook). 

Next, we will focus on **Instruction LLaMA**, a version of LLaMA fine-tuned to better understand and follow user instructions. 

We will use Llama 3.2 (released in September 2024). In particular, we will adopt the 1B version. On the scale of things, this model is on the smaller side, but it is still a very powerful model.

It has been released (along with a 3B version) with the intention of allowing running it on devices with modest hardware (e.g., mobile phones or other edge devices). 

In [3]:
model_id = "meta-llama/Llama-3.2-1B"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

We can use this model to generate text using the generate() method. We use random sampling (`do_sample=True`) and extract 5 samples (`num_return_sequences=5`). You can find other generation parameters [here](https://huggingface.co/docs/transformers/v4.46.0/en/main_classes/text_generation#transformers.GenerationConfig).

In [4]:
tokens = tokenizer("Hello, my name is", return_tensors="pt").to(model.device)
batch = model.generate(**tokens, do_sample=True, max_length=50, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id) # (assigning pad_token_id avoids a warning)
tokenizer.batch_decode(batch)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


['<|begin_of_text|>Hello, my name is Amy. I am a wife, mom of two boys, and a homeschooling mom of 5. I have been homeschooling since 2007, and have been teaching at The Home School Co-op since 201',
 '<|begin_of_text|>Hello, my name is Alex. I’m a 23-year-old male from the UK, currently living in the Netherlands. I’m a student, a software developer, and a gamer.\nI started gaming in 2009, when I got',
 '<|begin_of_text|>Hello, my name is Mandy. I am a certified Holistic Health Coach and I am passionate about helping people to take control of their health and well being. I believe that everyone has the ability to live a healthy and happy life and that',
 '<|begin_of_text|>Hello, my name is David and I am a photographer. I am based in the North of England and have been photographing weddings and events for over 15 years. I have a passion for capturing the moments in your special day and ensuring that',
 '<|begin_of_text|>Hello, my name is Sven and I am a PhD candidate in the Department 

### **Understanding the `tokenizer.chat_template`**

In this section, we will explore the **chat template** that is used to format and structure messages for a conversational assistant. The `tokenizer.chat_template` is a convenient way for organizing interactions between the user, system, and assistant in a way that the model can easily process and generate coherent responses.

### **What is a Chat Template?**

The chat template is a predefined format that ensures consistent structure for conversations. It marks the different roles in the interaction (system, user, assistant), and separates the various elements of the conversation using special tokens. This helps the language model understand which parts of the dialogue are instructions, which parts are user inputs, and where the assistant’s response should be generated.

Let's create an example of a possible (simplified) chat template:

In [5]:
import datetime

chat_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: """+datetime.datetime.now().strftime("%d %b %Y")+"""

{system_message}

<|eot_id|>
<|start_header_id|>user<|end_header_id|>

{user_message}

<|eot_id|>
"""

### **Hugging Face Pipeline Overview**

The **`pipeline`** method from Hugging Face’s Transformers library is a high-level API designed to streamline the process of using pre-trained models for a wide variety of **natural language processing (NLP) tasks**.

#### **What is a Pipeline?**

A pipeline is a modular tool that wraps around a pre-trained model, tokenizer, and task-specific configurations. It makes it easy to load and apply these models directly to different tasks, such as:
- **Text generation**
- **Text classification**
- **Question answering**
- **Summarization**
- **Translation**

By simply specifying the type of task (e.g., `"text-generation"`), `pipeline` takes care of loading and configuring a compatible model and tokenizer, providing a ready-to-use interface for generating results.

You can find a full list of supported pipelines on the [Hugging Face documentation](https://huggingface.co/docs/transformers/main_classes/pipelines).

In [6]:
# Create the pipeline with the model and tokenizer
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [7]:

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "What is 2 + 2?"},
]

# Format the messages using the chat template
formatted_messages = chat_template.format(
    system_message=messages[0]["content"],
    user_message=messages[1]["content"]
)


print(formatted_messages)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Oct 2024

You are a pirate chatbot who always responds in pirate speak!

<|eot_id|>
<|start_header_id|>user<|end_header_id|>

What is 2 + 2?

<|eot_id|>



Now, remember that for models to follow instruction tuning, they need to have been tuned on this kind of data. In this case, we are not using the instruction-tuned version. 

So, we can expect the model to produce a garbage response (it has never seen that kind of inputs before!). But let's try it anyway!

In [8]:
# Generate the output text 
outputs = pipe(
    formatted_messages,
    max_new_tokens=256,
    do_sample=True,
)

print(outputs[0]["generated_text"])


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Oct 2024

You are a pirate chatbot who always responds in pirate speak!

<|eot_id|>
<|start_header_id|>user<|end_header_id|>

What is 2 + 2?

<|eot_id|>
akedirs.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28
.readdirsystem.DateField: 2023-12-28

### **Differences Between Standard and Instruct Versions of Large Language Models (LLMs)**

Large Language Models (LLMs) come in different versions, with **standard** and **instruction-tuned (Instruct)** versions being the most common. Here’s a brief comparison:

#### **1. Purpose and Training**:
   - **Standard LLM**: The standard model is generally pre-trained on large datasets without specific instruction-following capabilities. Typically generates more open-ended responses, which can be useful for creative writing or general information retrieval where the response style is flexible.
   - **Instruct LLM**: Instruction-tuned models, like the **Llama-3.2 Instruct**, are fine-tuned on datasets designed to help the model understand and follow instructions effectively. This tuning enhances the model's ability to respond directly to user prompts and handle structured requests. It is fine-tuned to produce concise, direct responses that are often more relevant in task-specific or conversational AI applications.

Let's compare the outputs of the standard and Instruct versions of LLaMA to see the differences in their responses.

In [9]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-3.2-1B-Instruct"


# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = chat_template

# Create the pipeline with the model and tokenizer
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "What is 2 + 2?"},
]

# Format the messages using the chat template
formatted_messages = chat_template.format(
    system_message=messages[0]["content"],
    user_message=messages[1]["content"]
)

outputs = pipe(
    formatted_messages,
    max_new_tokens=512,
)

print(outputs[0]["generated_text"])


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Oct 2024

You are a pirate chatbot who always responds in pirate speak!

<|eot_id|>
<|start_header_id|>user<|end_header_id|>

What is 2 + 2?

<|eot_id|>
Arrr, me hearty! Ye be wantin' to know the answer to that ol' math problem, eh? Alright then, matey, let's set sail fer a swashbucklin' solution! *pours grog*

Alright, 2 + 2 be equal to... *taps foot impatiently*...4, me hearty! Ye landlubbers be thinkin' ye can solve it, but I be knowin' better! *winks*

Now, what be yer next question, matey?


### **Evaluation of the Tokenizer Chat Template**

Actually, the chat template of `meta-llama/Llama-3.2-1B-Instruct` is much more complex than the example above. It includes various components that help the model understand the context of the conversation, manage dates, handle tools, and structure messages effectively.

The template is written in [jinja](https://jinja.palletsprojects.com/en/stable/templates/), a language that allows for the dynamic generation of content based on variables, conditions and loops.


Let's print it and analyze its key components:
 

#### **Key Components of the Template**:
1. **System Message Extraction**:
   - The system message is extracted if the first role in the message list is labeled "system." This allows the template to clearly differentiate between user queries and system instructions.
   - If a system message exists, it is added to the template between special tokens (`<|start_header_id|>` and `<|end_header_id|>`), ensuring that the model knows when the system message starts and ends.

2. **Date Management**:
   - The template automatically handles the current date using either a provided `strftime_now` function or a default date (`"26 Jul 2024"`). This can be useful when the model needs to be aware of the date in contexts such as time-sensitive responses.

3. **Handling Tools**:
   - The template checks if **tools** are defined. If tools are available, it includes a description of these tools in the system message or the user message, depending on where they need to appear.
   - If the tools are part of the user message, the template ensures that the first user message prompts the user to respond in a structured format, such as using JSON for function calls.

4. **Message Processing**:
   - The template loops through the list of messages and processes each based on the role (`user`, `assistant`, `ipython`, or `tool`). It formats each message using start and end tokens for the roles, helping the model understand the structure of the conversation.
   - If the message involves tool calls, the template ensures that they are properly formatted into a structured JSON format to be passed back to the model for further processing.

5. **Ending the Assistant's Response**:
   - The template leaves a placeholder for the assistant’s response, which the model will generate during inference. This ensures that the assistant's response begins in the correct format, ready to be populated with the generated content.

#### **Why Is This Template Needed?**

- **Maintains Consistency**: This template ensures that the conversation is structured in a consistent manner, which is crucial for models designed to follow complex instructions or engage in multi-turn conversations.
- **Handles Tools**: By incorporating the ability to dynamically introduce tools and functionality, the template allows the model to expand beyond simple text-based conversations and perform function-based tasks.
- **Structured Outputs for Tools**: When the conversation involves tool calls (e.g., through APIs or function calls), the template ensures that these interactions are formatted properly for execution.

In [10]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

print(tokenizer.chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "26 Jul 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- 

Let's generate again the same example using the `chat_template` of `meta-llama/Llama-3.2-1B-Instruct` and analyze the output.

With a tokenizer that supports the chat template, we can directly call the `apply_chat_template()` method to convert a list of messages (each one a dictionary in the already discussed format) into a prompt.

Notice that, since we are not using any particular tools or other functionalities, our template will be similar to the one we manually introduced earlier.

In [11]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
input_tokens = tokenizer.apply_chat_template(messages)
print(tokenizer.decode(input_tokens))

outputs = pipe(
    messages,
    max_new_tokens=256,
)

# we are getting back the full conversation history
# as a list of messages outputs[0]["generated_text"]
# -1 : last message (assistant response)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Oct 2024

You are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|>


Arrr, me hearty! Yer lookin' fer a chatbot like meself, eh? Well, I be Captain Code, the swashbucklin' chatbot with a penchant fer pirate speak and a heart o' gold. I be here to answer yer questions, share me knowledge, and maybe even teach ye a thing or two about the high seas... er, I mean, the world o' knowledge.

So, hoist the sails and set course fer a treasure trove o' info, me matey! What be bringin' ye to these waters?


Notice that the pipeline already supports chat mode, so we can pass the list of messages (as long as they contain role/content keys) directly to the pipeline.

Alternatively, we could have passed the prompt as a string. In this case, however, we would have to manually extract the output from the model and parse it back.

In [12]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_tokens = tokenizer.apply_chat_template(messages)
prompt_string = tokenizer.decode(input_tokens)

outputs = pipe(
    prompt_string,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 28 Oct 2024

You are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|>assistant

Arrrr, ye be wantin' to know who I be? Well, matey, I be a swashbucklin' chatbot, here to swab the seven seas o' knowledge and answer yer questions to the best o' me abilities! Me name be "PirateBot", and I be a treasure trove o' info at yer disposal. So hoist the sails and set course fer a fine day o' learnin' and adventure! What be bringin' ye to these fair waters?


# 2. Mistral

In this part, we will explore the use of `Mistral-7B-Instruct-v0.2`developed by Mistral AI to generate structured responses in JSON format. 

In this exercise, we will generate random math questions and instruct Mistral-7B to respond in a structured JSON format. We will then save the responses to a JSON file and verify the answers programmatically. 

Let's first repeat the same example we did with LLaMA, but now using Mistral.


In [13]:
from transformers import pipeline

# Define the model ID
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
mistral_chat_template = tokenizer.chat_template

# Initialize the pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,  # Optimizes memory usage
    device_map="auto"            # Automatically distributes the model across available devices
)

# Define the message prompts for the conversation
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

# Generate the response
outputs = pipe(messages, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)

# Print the model's generated response
print(outputs[0]["generated_text"][-1]["content"])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


 Arr, I be Cap'n Parrotbeak, me hearty scallywag! How be thar, landlubber?

What brings ye to me virtual hideout? Come closer, but mind the plank!


1. **Generate random math questions:** Use Python to create questions with random numbers in a conversational style (e.g., “What is the sum of 245 and 173?”).

In [14]:
import random

def generate_random_math_questions(num_samples=5):
    templates = [
        "What is the sum of {} and {}?",
        "Can you add {} and {}?",
        "Calculate the sum of {} and {} for me.",
        "How much is {} plus {}?",
        "Please add {} and {}."
    ]
    
    questions = []
    for _ in range(num_samples):
        template = random.choice(templates)
        num1 = random.randint(10, 999)
        num2 = random.randint(10, 999)
        question = template.format(num1, num2)
        questions.append((question, num1, num2))  # store question with numbers for validation
    return questions


2. **Instruct the model to respond in JSON:** Use a system role instruction to ensure Mistral-7B answers in a JSON format containing the fields `num_1`, `num_2`, and `answer`. This makes the output compatible with automated processing or JSON parsers.

In [15]:
role_instruction = {
    "role": "system",
    "content": "Answer each question in JSON format with the fields 'num_1', 'num_2', and 'answer'. Provide only JSON to ensure compatibility with a JSON parser."
}


3. **Save and verify responses:** Generate and store model responses in a JSON file and check if the answers match expected values.

In [16]:
import json
from tqdm import tqdm

# Generate questions and answers, then save to JSON
questions = generate_random_math_questions(num_samples=5)
answers = []

# Generate structured answers for each question
for question, num1, num2 in tqdm(questions):

    # Define the message prompts
    formatted_messages = [
        role_instruction,
        {"role": "user", "content": question}
    ]
    
    # Generate the response
    outputs = pipe(formatted_messages, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    
    # Extract the model's JSON output
    structured_answer = outputs[0]["generated_text"]

    answers.append({
        "question": question,
        "num_1": num1,
        "num_2": num2,
        "model_answer": structured_answer
    })

# Save answers to a JSON file
with open("model_answers.json", "w") as f:
    json.dump(answers, f, indent=2)


100%|██████████| 5/5 [00:07<00:00,  1.56s/it]


In [17]:
import json

# Function to parse model's answer and verify correctness
def verify_answer(entry):
    try:
        # Extract expected values
        num1, num2 = entry["num_1"], entry["num_2"]
        expected_answer = num1 + num2
        
        # Extract the assistant's response from the list of messages
        assistant_message = next(
            (msg["content"] for msg in entry["model_answer"] if msg["role"] == "assistant"), None
        )
        
        if assistant_message is None:
            raise ValueError("Assistant's message not found in model_answer")
        
        # Parse model's structured answer from JSON
        model_response = json.loads(assistant_message.strip())  # Ensure model_answer is a string
        
        print(f"Expected answer: {num1} + {num2} = {expected_answer}")
        print(f"Model's answer: {model_response['num_1']} + {model_response['num_2']} = {model_response['answer']}")
        
        # Check if the values match
        if (model_response["num_1"] == num1 and 
            model_response["num_2"] == num2 and 
            model_response["answer"] == expected_answer):
            return True
        else:
            return False
    except (json.JSONDecodeError, KeyError, TypeError, ValueError) as e:
        # Handle cases where parsing fails or keys are missing
        print(f"Error verifying entry: {entry}. Error: {e}")
        return False

# Load answers from the JSON file and verify
try:
    with open("model_answers.json", "r") as f:
        saved_answers = json.load(f)
except (json.JSONDecodeError, FileNotFoundError) as e:
    print(f"Error loading JSON file: {e}")
    saved_answers = []

for i, entry in enumerate(saved_answers, 1):
    result = verify_answer(entry)
    print(f"Question {i}:", "Correct" if result else "Incorrect", "\n")

Expected answer: 276 + 597 = 873
Model's answer: 276 + 597 = 873
Question 1: Correct 

Expected answer: 747 + 589 = 1336
Model's answer: 747 + 589 = 1336
Question 2: Correct 

Expected answer: 958 + 907 = 1865
Model's answer: 958 + 907 = 1865
Question 3: Correct 

Expected answer: 323 + 73 = 396
Model's answer: 323 + 73 = 396
Question 4: Correct 

Expected answer: 711 + 801 = 1512
Model's answer: 711 + 801 = 1512
Question 5: Correct 

